# Wordle Solver

In [1]:
from wordle.utils import Wordle

%load_ext autoreload
%autoreload 2

### Play Wordle

In [2]:
# Initialize Wordle class
wordle = Wordle()

# Main function that returns a list of possible words
green_letters = "ny"
green_positions = [3, 5]
yellow_letters = "un"
yellow_positions = [3, 4]
gray_letters = "tiaresolbha"

# Find words in a list of possible words
words = wordle.find_words(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    gray_letters,
    answer_word_list=True,
)

# Choose a word to play
wordle.choose_word_to_play(word_threshold=2)

Number of possible words: 1
funky 0.0


In [3]:
# Repetitive letters
letters_df = wordle.repetitive_letters()
print(letters_df.to_string(index=False))

Letters  Count
      F      1
      U      1
      N      1
      K      1
      Y      1


In [4]:
df = wordle.load_data()
df

,Names,Games_Won
0,Murilo,323
1,Barbara,140
2,Draw,319


In [5]:
# Update score
score_df = wordle.score()
score_df

,Names,Games_Won
0,Murilo,324
1,Barbara,140
2,Draw,319


In [6]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        324
Barbara        140
   Draw        319


#### Git commit message

In [7]:
# git commit message
commit_message = wordle.get_commit_message()
print(commit_message)

git commit -m "updated wordle 324-140-319"


#### Set custom score

In [8]:
# help(wordle.reset_score)

In [9]:
# help(wordle.set_score)

In [10]:
# # Set custom score
# df = wordle.set_score(m_score=311, b_score=132, draw_score=308)
# df

#### Reset score

In [11]:
## Reset score
# wordle.reset_score()